# 320. Custom Dataset을 이용한 Hugging Face BERT model Fine Tuning

- NAVER Movie review dataset을 이용하여 transformers BERT model을 fine tuning  → 감성분석 모델 작성

- Pytorch 와 Trainer를 이용한 Fine Tuning (Pytorch version이 Tensorflow 보다 안정적)

- Colab gpu 사용

In [2]:
# Hugging Face Transformers에서 BERT 토크나이저 로드
from transformers import BertTokenizer
# Hugging Face Transformers에서 BERT 기반 문장 분류 모델 로드
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch.nn.functional as F
import tensorflow as tf
import torch
import pandas as pd

In [3]:
DATA_TRAIN_PATH = tf.keras.utils.get_file("ratings_train.txt",
                     "https://raw.github.com/ironmanciti/Infran_NLP/master/data/naver_movie/ratings_train.txt")
DATA_TEST_PATH = tf.keras.utils.get_file("ratings_test.txt",
                    "https://raw.github.com/ironmanciti/Infran_NLP/master/data/naver_movie/ratings_test.txt")

### Train Set

In [4]:
#  학습 데이터 로드
train_data = pd.read_csv(DATA_TRAIN_PATH, delimiter='\t')

print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
# 결측값(NaN)이 포함된 행을 모두 제거
train_data.dropna(inplace=True)

# 현재 DataFrame의 구조 요약 출력
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


### Test Set

In [6]:
#  검증 데이터 로드
test_data = pd.read_csv(DATA_TEST_PATH, delimiter='\t')

print(test_data.shape)
test_data.head()

(50000, 3)


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [7]:
# 결측값(NaN)이 포함된 행을 모두 제거
test_data.dropna(inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- 훈련 시간 단축을 위해 data size 축소

In [8]:
# 훈련 데이터에서 무작위로 100,000개 샘플 추출 (재현성을 위해 random_state 고정)
df_train = train_data.sample(n=15_000, random_state=1)

# 테스트 데이터에서 무작위로 10,000개 샘플 추출
df_test = test_data.sample(n=5_000, random_state=1)

# 추출된 데이터프레임의 행과 열 크기 출력
print(df_train.shape)
print(df_test.shape)

(15000, 3)
(5000, 3)


In [9]:
# 훈련 데이터의 'label' 열에 있는 각 클래스(레이블)별 개수를 집계
df_train['label'].value_counts()

,count
label,
0,7524
1,7476


In [10]:
# 훈련 데이터에서 입력 문장(document)과 레이블(label)을 리스트로 추출
X_train = df_train['document'].values.tolist()      # 입력 텍스트 (리스트 형태)
y_train = df_train['label'].values.tolist()               # 정답 레이블 (리스트 형태)

# 테스트 데이터에서도 동일하게 입력과 레이블을 리스트로 추출
X_test = df_test['document'].values.tolist()    # 입력 텍스트 (리스트 형태)
y_test = df_test['label'].values.tolist()             # 정답 레이블 (리스트 형태)

## pre-trained bert model 호출
### tokenizer 호출
- 토큰화 처리를 합니다. bert 다국어 version 용의 pre-trained tokenizer 를 불러옵니다.

In [11]:
# 사전학습된 BERT 토크나이저 불러오기
# 'bert-base-multilingual-cased'는 100개 이상의 언어를 지원하는 다국어 BERT 모델로,
# 대소문자 구분(cased)을 유지함
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pre-trained tokenizer 를 이용하여 train set 과 test set 을 token 화 합니다.

- Input IDs : 토큰 인덱스, 모델에서 입력으로 사용할 시퀀스를 구축하는 토큰의 숫자 표현
- Token Type IDs : 한 쌍의 문장 또는 질문 답변에 대한 분류 시 사용  
- attention mask : `1`은 주목해야 하는 값을 나타내고 `0`은 패딩된 값을 나타냅니다.  
```
[CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP]
ex) [CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
```

In [12]:
# 훈련 데이터(X_train)를 BERT 입력 형식에 맞게 토크나이즈
# - truncation=True: 최대 길이를 초과하는 문장은 자동으로 자름
# - padding=True: 짧은 문장은 최대 길이에 맞춰 0으로 패딩
train_encodings = tokenizer(X_train, truncation=True, padding=True)

# 테스트 데이터(X_test)도 동일한 방식으로 토크나이즈
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [13]:
# 토크나이징된 훈련 데이터의 키 목록 확인
# 일반적으로 'input_ids', 'attention_mask', (선택적으로 'token_type_ids')가 포함됨
print(type(train_encodings))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [14]:
print(train_encodings['input_ids'][0])
print(train_encodings['attention_mask'][0])
print(train_encodings['token_type_ids'][0])

[101, 113, 9926, 34907, 20626, 58931, 24974, 122, 114, 9532, 25503, 12030, 28911, 9367, 19855, 47869, 9682, 9634, 21386, 136, 8924, 11261, 119351, 12605, 20308, 12453, 117, 9792, 73352, 21876, 20173, 9294, 36553, 11287, 52560, 9391, 11664, 9640, 18784, 12030, 12508, 9304, 12508, 19709, 9684, 52560, 10892, 8932, 118651, 14523, 48549, 119, 8905, 119377, 11102, 117, 9604, 78123, 11102, 117, 9684, 89523, 42769, 15387, 9792, 73352, 21876, 20173, 47058, 8982, 28188, 11664, 9294, 36553, 11287, 52560, 9597, 10530, 19709, 9792, 73352, 21876, 100698, 11018, 9670, 14871, 15387, 9637, 12945, 22333, 43022, 113, 9069, 18227, 114, 63783, 9641, 42337, 14801, 119, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Convert encodings to Tensors

- 레이블과 인코딩을 Dataset 개체로 변환합니다. Pytorch를 이용합니다.  

- PyTorch에서 이것은 `torch.utils.data.Dataset` 객체를 하고 `__len__` 및 `__getitem__`을 구현하여 수행됩니다.


In [15]:
# PyTorch Dataset 클래스를 상속하여 IMDb 감성 분석용 커스텀 데이터셋 정의
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        # 토크나이즈된 입력 (input_ids, attention_mask 등) 저장
        self.encodings = encodings
        # 정답 레이블 (선택사항)
        self.labels = labels

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터 추출
        # encodings 딕셔너리에서 각 항목별로 같은 인덱스를 추출하고 텐서로 변환
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # 레이블이 있는 경우 함께 반환
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        # 데이터셋의 전체 샘플 수 반환
        return len(self.encodings["input_ids"])

# 훈련용 PyTorch Dataset 객체 생성
train_dataset = IMDbDataset(train_encodings, y_train)

# 테스트용 PyTorch Dataset 객체 생성
test_dataset = IMDbDataset(test_encodings, y_test)

이제 데이터 세트가 준비되었으므로 🤗 `Trainer` 또는 기본 PyTorch/TensorFlow를 사용하여 모델을 미세 조정할 수 있습니다. [training](https://huggingface.co/transformers/training.html)을 참조하세요.

- Training warmup steps :  

    - 이는 일반적으로 설정된 수의 훈련 단계(워밍업 단계)에 대해 매우 낮은 학습률을 사용한다는 것을 의미합니다. 워밍업 단계 후에 "일반" 학습률 또는 학습률 스케줄러를 사용합니다. 또한 워밍업 단계 수에 따라 학습률을 점진적으로 높일 수 있습니다.

- weight_decay : 가중치 감쇠. L2 regularization

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='./results',               # 모델 출력 결과(가중치 등)를 저장할 디렉토리
    num_train_epochs=2,                   # 학습 전체 epoch 수
    per_device_train_batch_size=8,        # 학습 시 디바이스(GPU/CPU)당 배치 크기
    per_device_eval_batch_size=16,        # 평가 시 디바이스당 배치 크기
    warmup_steps=500,                     # 러닝레이트 스케줄러를 위한 워밍업 스텝 수
    weight_decay=0.01,                    # 가중치 감쇠(정규화) 계수
    logging_dir='./logs',                 # 로그 저장 디렉토리
    logging_steps=100,                    # 몇 스텝마다 로그를 출력할지 설정
    report_to=None,                       # wandb 비활성화
    run_name='naver_movie_sentiment'      # 명시적 run_name 설정
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### model Train
- Xet Storage는 Hugging Face에서 도입한 고속 버전 관리 + 스토리지 시스템으로,
모델과 데이터 파일을 효율적으로 다운로드/업로드/버전 관리하기 위한 기술입니다.  
- wandb는 Weights & Biases의 약자로, 머신러닝 및 딥러닝 프로젝트의 학습 과정을 시각화·추적·관리할 수 있게 해주는 도구입니다.
대표적으로 Hugging Face Trainer, PyTorch, TensorFlow 등과 쉽게 연동됩니다.

Colab 에서 약 17분 소요

In [17]:
import time

# 사전학습된 다국어 BERT 모델 로드 (문장 분류용으로 head가 붙어 있음)
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=2  # 긍정/부정 2개 클래스 명시적 지정
)

# Hugging Face의 Trainer 객체 생성
trainer = Trainer(
    model=model,                  # 학습할 모델
    args=training_args,           # 학습 설정 (TrainingArguments 객체)
    train_dataset=train_dataset,  # 훈련 데이터셋
    eval_dataset=test_dataset     # 평가 데이터셋
)

# 학습 시작 시간 기록
s = time.time()

# 모델 학습 수행
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,0.697700
200,0.679800
300,0.647500
400,0.631900
500,0.625700
600,0.697100
700,0.665400
800,0.597400
900,0.593300
1000,0.693200


TrainOutput(global_step=3750, training_loss=0.5670912694295247, metrics={'train_runtime': 1525.6148, 'train_samples_per_second': 19.664, 'train_steps_per_second': 2.458, 'total_flos': 1896249598200000.0, 'train_loss': 0.5670912694295247, 'epoch': 2.0})

In [18]:
print("경과 시간 : {:.2f}분".format((time.time() - s)/60))

경과 시간 : 25.44분


In [19]:
# 테스트 데이터셋을 사용하여 모델 성능 평가
# 반환값에는 손실(loss), 정확도(accuracy) 등의 평가 지표가 포함됨
trainer.evaluate(test_dataset)

{'eval_loss': 0.4810846149921417,
 'eval_runtime': 34.6963,
 'eval_samples_per_second': 144.107,
 'eval_steps_per_second': 9.021,
 'epoch': 2.0}

In [20]:
# 테스트 데이터셋에 대해 예측 수행
# 출력은 예측 결과(predictions), 실제 정답(label_ids), 평가 지표(metrics)를 포함한 객체
prediction = trainer.predict(test_dataset)
prediction

PredictionOutput(predictions=array([[ 1.4017929 , -1.3930826 ],
       [ 1.2493985 , -1.319022  ],
       [-0.31896353, -0.13290049],
       ...,
       [-1.0467689 ,  1.191216  ],
       [ 1.2977395 , -1.3431832 ],
       [-0.46728617,  0.31850854]], dtype=float32), label_ids=array([0, 0, 1, ..., 1, 1, 0]), metrics={'test_loss': 0.4810846149921417, 'test_runtime': 32.7265, 'test_samples_per_second': 152.781, 'test_steps_per_second': 9.564})

fine-tuned model 은 logit 을 return

In [21]:
# 현재 Trainer에 포함된 모델에서 분류기(classifier) 층 확인
# 이 층은 BERT 출력(hidden state)을 받아 최종 분류 결과를 계산하는 레이어
trainer.model.classifier

Linear(in_features=768, out_features=2, bias=True)

In [22]:
# 모델 예측 결과에서 로짓(logits) 값을 텐서로 변환
# prediction[0]은 trainer.predict()의 결과 중 'predictions' (로짓 값)
y_logit = torch.tensor(prediction[0])

# 처음 10개 샘플의 로짓 출력
# 각 샘플마다 클래스 수만큼의 점수(예: 2-class 분류면 [logit0, logit1])가 있음
y_logit[:10]

tensor([[ 1.4018, -1.3931],
        [ 1.2494, -1.3190],
        [-0.3190, -0.1329],
        [ 1.4652, -1.4232],
        [-1.0382,  1.1794],
        [ 1.4302, -1.4073],
        [-0.2727, -0.1678],
        [ 1.4225, -1.4033],
        [-1.0468,  1.1910],
        [ 0.3616, -0.6903]])

In [23]:
# 소프트맥스 함수를 사용해 각 샘플의 클래스별 확률을 계산
# dim=-1: 마지막 차원(클래스 차원) 기준으로 소프트맥스 적용
# argmax(axis=1): 확률이 가장 높은 클래스의 인덱스를 예측값으로 선택
# numpy(): PyTorch 텐서를 넘파이 배열로 변환
y_pred = F.softmax(y_logit, dim=-1).argmax(axis=1).numpy()

# 예측된 레이블 중 앞 30개를 리스트로 출력
print(list(y_pred[:30]))

# 실제 정답 레이블(y_test) 중 앞 30개를 출력
print(y_test[:30])

[np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1)]
[0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]


In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score

# 예측값과 실제 정답 사이의 정확도(accuracy)를 계산
print(accuracy_score(y_test, y_pred))

# 혼동 행렬(confusion matrix) 계산
# 실제 레이블과 예측 레이블을 비교하여 각 클래스별 예측 결과를 표로 요약
cm = confusion_matrix(y_test, y_pred)
cm

0.7856


array([[1825,  670],
       [ 402, 2103]])

In [26]:
# 예측할 문장
x = "돈주고 보기에는 아까운 영화 ㅠㅠ..."
# x = "내 인생 최고 명작"

# 1. 입력 토크나이즈
inputs = tokenizer([x], truncation=True, padding=True, return_tensors="pt")

# 2. 입력을 GPU로 이동하고 예측
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}  # 모든 입력을 같은 디바이스로 이동

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 3. 소프트맥스 → 확률 → argmax
probs = F.softmax(logits, dim=-1)
pred = torch.argmax(probs, dim=1).item()

# 4. 결과 출력
print("긍정" if pred == 1 else "부정")

긍정


# Next Step
20 만개 전체 dataset으로 fine tuning